# Exploración del conjunto de datos
En este notebook exploraremos los datos para poder tomar una decisión sobre el diseño de la solución. En cada sección encontrarás preguntas con la intención de motivar una reflexión sobre su diseño. Mientras que no necesitan ser respondidas explícitamente, les invito a que las utilicen como guía al momento de justificar sus decisiones de diseño y presentar sus resultados.

## Datos de entrenamiento
En el conjunto de datos tenemos información sobre las *compras* realizadas en una tienda de ropa en linea. En la siguiente celda podemos ver las columnas o atributos que tenemos a nuestra disposición.
- ¿Tenemos valores faltantes o todas las columans tienen la información completa?
- ¿Qué tipos de columnas tenemos y cómo podemos procesarlas?

In [ ]:
import pandas as pd
import os
train_file = "../../datasets/customer_purchases/customer_purchases_train.csv"
train_file = os.path.abspath(train_file)
train_df = pd.read_csv(train_file)

train_df.info()
train_df.columns


## Prueba
Recuerda que en prueba queremos saber si un cliente existente comprará un produco *no existente*. En el conjunto anterior tenemos columnas que solo tendrán información si se ha generado una compra, ¿Que columnas son estas? ¿Que valores podríamos esperar de las mismas para productos que no han salido a la venta? ¿Tiene sentido usarlas en entrenamiento tal cual? ¿Podemos extraer información valiosa de las mismas?

In [ ]:
import pandas as pd
import os
test_file = "../../datasets/customer_purchases/customer_purchases_test.csv"
test_file = os.path.abspath(test_file)
test_df = pd.read_csv(test_file)

test_df.info()
test_df.columns

# Visualizar los datos por etiqueta

Visualiza las etiquetas del conjunto de datos de entrenamiento

In [ ]:
import matplotlib.pyplot as plt

train_df["label"].value_counts().plot(
    kind="bar",
    xlabel="Label",
    ylabel="Cantidad purchase_id",
    title="purchase_id por etiqueta",
    figsize=(6,4)
)
plt.show()

¿Notas algo extraño?  ¿Podemos entrenar un algoritmo de ML en este dataset tal cual se nos presenta?
¿Qué problemas podríamos encontrarnos?

## Visualizar estadisticas por producto y por cliente

En el dataset solo tenemos información de las *compras*, es decir cada que se repite un `customer_id`, podemos saber cuantas compras ha hecho ese cliente. De la misma manera, contar cuantas veces se repite un `item_id` nos dice cuantas veces se ha comprado ese producto. Utiliza pandas para ver si existen clientes que compran que mayor frecuencia que otros, o productos que se compran mas que otros. Responde:
- ¿Consideras que los datos se encuentran balanceados dada la información anterior?
- ¿Qué estrategías podrian ser útiles al momento de generar negativos?

### Datos por usuario

In [ ]:
# Conteo de compras por cliente
by_customer = train_df["customer_id"].value_counts()
print(by_customer.describe())


¿Cómo podemos interpretar la información anterior?

Visualicemos ahora en una gráfica

In [ ]:
max_val = by_customer.max()
nbuckets = 7
increment = max_val // nbuckets

bins = []
labels = []
bins = list(range(0, max_val + increment, increment))
labels = [f"{bins[i]+1}-{bins[i+1]}" for i in range(len(bins)-1)]


customer_by_purchase = train_df['customer_id'].value_counts()
customer_buckets = pd.cut(customer_by_purchase, bins=bins, labels=labels)
bucket_counts = customer_buckets.value_counts().sort_index()

bucket_counts.plot(kind="bar", figsize=(6,4))
plt.title("Clientes agrupados por número de compras")
plt.xlabel("Rango de compras")
plt.ylabel("Cantidad de clientes")
plt.show()

### Datos por item

In [ ]:
# Conteo de compras por producto
by_item = train_df["item_id"].value_counts()
print(by_item.describe())


In [ ]:
max_val = by_item.max()
nbuckets = 5
increment = max_val // nbuckets

bins = []
labels = []
bins = list(range(0, max_val + increment, increment))
labels = [f"{bins[i]+1}-{bins[i+1]}" for i in range(len(bins)-1)]


items_by_purchase = train_df['item_id'].value_counts()
item_buckets = pd.cut(items_by_purchase, bins=bins, labels=labels)
item_counts = item_buckets.value_counts().sort_index()

item_counts.plot(kind="bar", figsize=(6,4))
plt.title("Productos agrupadas por cantidad de compras")
plt.xlabel("Rango de compras")
plt.ylabel("Cantidad de clientes")
plt.show()

# Generación ejemplos negativos

Como pudiste notar solo tenemos información de compras. Si se deseara realizar un algoritmo de clasificación para productos nuevos, este fallaría al ser entrneado en el dataset original ya que no tendría información de ejemplos negativos. Por lo anterior, este tipo de problemas comunmente se modela como sistemas de recomendación. 

Otra manera de resolverlo, es generar ejemplos negativos dada la información en el conjunto, por ejemplo sabemos el inventario completo (los `item_id` únicos) y sus atributos, asi como el conjunto completo de clientes (`customer_id` únicos) y para cada cliente sabemos los productos que han comprado, *pero tambien sabemos los que no compraron* si al conjunto de productos completo, le restamos el conjunto de productos que si compró. Esto nos da una señal de ejemplos negativos que podemos usar para clasificación.

Dicho esto tienes 2 opciones:
1. Investigar sistemas de recomendación tradicionales [(collaborative filtering, content-based filtering etc)](https://www.geeksforgeeks.org/machine-learning/content-based-vs-collaborative-filtering-difference/)
2. Modelaro como un sistema de clasificación y generar tus propios ejemplos negativos

Son libres de elegir la decisión que prefieran.

En el caso de clasificación tenemos que considerar lo siguiente: Para muchos clientes, el conjunto de items no comprados será mucho mayor que el de items comprados. 
- ¿Qué problemas puedo tener si elijo como negativos todos los no comprados para todos los clientes?
- ¿Pueden pensar en diferentes estrategias para generar los negativos?

Pueden utilizar el archivo [negative_generation.py](./negative_generation.py) para implementar diferentes estrategias de generación de negativos. 

In [ ]:
from ml_clases.proyectos.P01_customer_purchases.negative_generation import gen_all_negatives, gen_random_negatives

train_df_neg = gen_random_negatives(train_df,n_per_positive=1)

train_df_full = pd.concat((train_df, train_df_neg))
# Shuffle para que cuando hagamos el split tengamos de ambos muestras
train_df_full = train_df_full.sample(frac=1) # 100%

train_df_full["label"].value_counts().plot(
    kind="bar",
    xlabel="Label",
    ylabel="Cantidad purchase_id",
    title="purchase_id por etiqueta",
    figsize=(6,4)
)
plt.show()

- ¿Que podemos observar cuando generamos todos los negativos?
- ¿Se les ocurre otra manera de elegirlos?

In [ ]:
import seaborn as sns
sns.pairplot(train_df_full)

## Entregables: Análisis exploratorio
Dentro de sus entregables de analisis, intenten responder a las preguntas:
- ¿Existen valores faltantes (i.e. NaN / None)? ¿Como proponen manejarlos?
- ¿Que atributos pueden ser predictores útiles?
- ¿Tenemos etiquetas balanceadas? ¿Cómo van a manejarlas?
- ¿Que cantidad de productos existententes vs. nuevos tenemos?
- ¿Cuantos clientes tenemos?
- ¿Que tipos de datos tenemos y como podemos procesarlos?
- ¿Existen correlaciones fuertes entre variables dependientes?
- ¿Que correlaciones pueden ver entre las variables dependientes y la independiente?

### Bonus:
Se otorgarán puntos extra a análisis exploratorios excepcionales que integren diferentes métodos de ML visualizaciones y reconocimientos de patrones más complejos que vayan mas allá de lo propuesto/observado en este notebook.

### Recomendaciones generales
- Les recomiendo explorar la libreria de [seaborn](https://seaborn.pydata.org/) y las diferentes opciones de visualización que tienen.
- Los resultados de este problema dependen en gran medida de los atributos derivados y manejo de datos por lo que les recomiento invertir la mayoría de su tiempo a explorar entrenamiento con diferentes conjuntos de atributos.

## Extracción de atributos

Podemos extraer atributos del cliente para utilizar tanto en entrenamiento como validación o prueba. utilicen las celdas de abajo para lo mismo y guardar los datos en `customer_feat.csv`. Posteriormente podemos usar estos atributos junto con los de los items existentes asi como nuevos para entrenar y posteriormente predecir.

Puedes reemplazar tu eleccion de extracción de atributos en [data_processing.py -> extract_customer_features](./data_processing.py)